## Chapter 04-03 近代暗号

In [ ]:
# Collaboratoryを使っている方は，下の行のコメント(#)を取り除いて実行してください
#pip install minbc_lib

### 排他的論理和(XOR)と暗号

In [ ]:
bin(15)

In [ ]:
bin(0b1111 ^ 0b1010)

In [ ]:
bin(0b101 ^ 0b1010)

### バーナム暗号

In [ ]:
def binary_cipher(plist, key):
    # XORを使って数のリストを暗号化する
    result = []   # 結果を保存するリストを初期化
    for p, k in zip(plist, key):  # 二つのリストを使ってループ
        result.append(p ^ k)   # XORの結果をリストに追加
    return result

In [ ]:
from minbc_lib.bcipher import *

In [ ]:
plist = [65, 66, 67]
key = [1, 2, 3]
clist = binary_cipher(plist, key)
clist    # 暗号化の結果を表示

In [ ]:
clist2 = binary_cipher(clist, key)
clist2    # 復号化の結果を表示

### 乱数の生成とストリーム暗号

In [ ]:
from minbc_lib.bcipher import *

In [ ]:
def KSA(key):
    # 0から255までのリストを作りシャッフルする
    k = list(range(256))  # 0から255までのリストを作る
    j = 0   # 入れ替えに使うインデックスを初期化
    for i in range(256): # 0から255までループ
        # 入れ替えに使うインデックスを計算
        j = (j+k[i]+ key[i % len(key)]) % 256
        k[i], k[j] = k[j], k[i]  # リストを入れ替える
    return k

In [ ]:
k = KSA([1, 2, 3])

In [ ]:
def PRGA(k, l):
    # 0から255までの疑似乱数をl個生成する
    i = 0
    j = 0 # 入れ替えに使うインデックスを初期化
    key = []  # キーストリーム用のリストを初期化
    for c in range(l):
        i = (i + 1) % 256
        j = (j + k[i]) % 256  # 入れ替え用のインデックスを計算
        k[i], k[j] = k[j], k[i]  # リストを入れ替える
        key.append(k[(k[i] + k[j]) % 256])
    return key

In [ ]:
keystream = PRGA(k, 3)   # 長さ3のキーストリームを作る

In [ ]:
plist = [65, 66, 67]    # 平文
clist = binary_cipher(plist, keystream)  # XORで暗号化
clist  # 暗号化した結果を表示

In [ ]:
clist2 = binary_cipher(clist, keystream)  # 復号化
clist2  # 復号化した結果を表示

### 乱数のランダムさを調べる

In [ ]:
k = KSA([1, 2, 3])     # KSAのリストを作る
key = PRGA(k, 1000000)  # キーストリームを作る

In [ ]:
from minbc_lib.bcipher import *       # ライブラリをインポート
show_rc4_randdomness(key)   # グラフを描画する

In [ ]:
from random import randint  # 乱数を作る関数をインポート
# ランダムなKSAのキーを使って，RC4のキーストリームを10万回作る
rc4_ks = []  # RC4のキーストリームを保存するリスト
for i in range(100000):
    # ランダムな3つの値を持つKSAの鍵を作る
    ksa_key = [randint(0, 255), randint(0, 255), randint(0, 255)]
    k = KSA(ksa_key)   # KSAのリストを作る
    key = PRGA(k, 10)  # 長さ10のキーストリームを作る
    rc4_ks.append(key)  # キートスリームをリストに追加

In [ ]:
# キーストリームの二番目に出現する数を集計する
second_keys = []  # 二番目の数を保存するリストを初期化
for i in range(100000):
    # キーストリームの二番目を取り出してリストに追加
    second_keys.append(rc4_ks[i][1])

In [ ]:
second_keys.count(0)

In [ ]:
second_keys.count(1)

In [ ]:
show_rc4_randdomness(second_keys)

### ブロック暗号

In [ ]:
def p(num):
    # 4ビットの数を転置する
    res = (num&1)<<3 | (num&2)<<1 | (num&4)>>2 | (num&8)>>2
    return res

In [ ]:
from minbc_lib.bcipher import *

In [ ]:
p(0b1010)

In [ ]:
def r_p(num):
    # 4ビットの数を転置する(逆)
    res = (num&1)<<2 | (num&2)<<2 | (num&4)>>1 | (num&8)>>3
    return res

In [ ]:
r_p(6)

In [ ]:
# 換字用のリスト
sbox = [12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2]

def s(num):
    # 換字を行う関数
    return sbox[num]

def r_s(num):
    # 換字を行う関数(逆)
    return sbox.index(num)

In [ ]:
r_s(s(10))

In [ ]:
def simple_bcipher(num, key):
    # ブロック暗号で暗号化する
    r = s(num)  # 転置
    r = r ^ key # ラウンド鍵でXOR
    r = p(r)      # 換字
    return r

def r_simple_bcipher(num, key):
    # ブロック暗号で復号化する
    r = r_p(num)      # 換字
    r = r ^ key # ラウンド鍵でXOR
    r = r_s(r)  # 転置
    return r

In [ ]:
num = 5    # 平文となる数
for i in range(5): # 5ラウンド繰り返す
    # ラウンド鍵10で暗号化
    num = simple_bcipher(num, 10)
num   # 暗号化した結果を表示

In [ ]:
num = 0    # 暗号文
for i in range(5): # 10ラウンド繰り返す
    # ラウンド鍵10で暗号化
    num = r_simple_bcipher(num, 10)
num   # 複合化した結果を表示